Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function

import os
import sys
import tarfile

import imageio
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from six.moves import cPickle as pickle
from six.moves.urllib.request import urlretrieve

# Config the matplotlib backend as plotting inline in IPython
#%matplotlib inline


First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labeled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [3]:
url = 'https://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None
data_root = '.'  # Change me to store data elsewhere

def download_progress_hook(count, blockSize, totalSize):
    """A hook to report the progress of a download. This is mostly intended for users with
    slow internet connections. Reports every 5% change in download progress.
    """
    global last_percent_reported
    percent = int(count * blockSize * 100 / totalSize)

    if last_percent_reported != percent:
        if percent % 5 == 0:
            sys.stdout.write("%s%%" % percent)
            sys.stdout.flush()
        else:
            sys.stdout.write(".")
            sys.stdout.flush()

        last_percent_reported = percent

def maybe_download(filename, expected_bytes, force=False):
    """Download a file if not present, and make sure it's the right size."""
    dest_filename = os.path.join(data_root, filename)
    if force or not os.path.exists(dest_filename):
        print('Attempting to download:', filename)
        filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
        print('\nDownload Complete!')
    statinfo = os.stat(dest_filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', dest_filename)
    else:
        raise Exception(
            'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
    return dest_filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)


Attempting to download: notMNIST_large.tar.gz


0%

.

.

.

...

.

.

.

100%


Download Complete!
Found and verified ./notMNIST_small.tar.gz


Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labeled A through J.

In [4]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

Extracting data for ./notMNIST_large. This may take a while. Please wait.


['./notMNIST_large/A', './notMNIST_large/B', './notMNIST_large/C', './notMNIST_large/D', './notMNIST_large/E', './notMNIST_large/F', './notMNIST_large/G', './notMNIST_large/H', './notMNIST_large/I', './notMNIST_large/J']
Extracting data for ./notMNIST_small. This may take a while. Please wait.


['./notMNIST_small/A', './notMNIST_small/B', './notMNIST_small/C', './notMNIST_small/D', './notMNIST_small/E', './notMNIST_small/F', './notMNIST_small/G', './notMNIST_small/H', './notMNIST_small/I', './notMNIST_small/J']


---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---


In [5]:
from IPython.display import display, Image

file_list_A=os.listdir(train_folders[0])

#for x in file_list_A: 
#file_path = os.path.join(train_folders[0],file_list_A[1] )
#print(file_path)
#Image(filename=file_path, format="png")

root=train_folders[0]
file_list_A=os.listdir(root)

for x in file_list_A: 
    file_path = os.path.join(root, x)
    display(Image(filename=file_path))
    break;
    

Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. 

A few images might not be readable, we'll just skip them.

In [6]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (imageio.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except (IOError, ValueError) as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

Pickling ./notMNIST_large/A.pickle.
./notMNIST_large/A


Could not read: ./notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.


Could not read: ./notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.


Could not read: ./notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.


Full dataset tensor: (52909, 28, 28)
Mean: -0.12825038
Standard deviation: 0.4431209


...

Full dataset tensor: (1872, 28, 28)
Mean: -0.11818536
Standard deviation: 0.45227855
Pickling ./notMNIST_small/G.pickle.
./notMNIST_small/G


Full dataset tensor: (1872, 28, 28)
Mean: -0.09255032
Standard deviation: 0.44900596
Pickling ./notMNIST_small/H.pickle.
./notMNIST_small/H


Full dataset tensor: (1872, 28, 28)
Mean: -0.058689233
Standard deviation: 0.45875892
Pickling ./notMNIST_small/I.pickle.
./notMNIST_small/I


Full dataset tensor: (1872, 28, 28)
Mean: 0.052645072
Standard deviation: 0.47189352
Pickling ./notMNIST_small/J.pickle.
./notMNIST_small/J


Full dataset tensor: (1872, 28, 28)
Mean: -0.1516892
Standard deviation: 0.44801357


---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---


In [7]:
import matplotlib.pyplot as plt

pickle_file = train_datasets[1]
with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        plt.imshow(letter_set[0])

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

In [8]:
for file in test_datasets:
    print(file)
    with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        print(len(letter_set))

./notMNIST_small/A.pickle


52911
./notMNIST_small/B.pickle


52911
./notMNIST_small/C.pickle


52911
./notMNIST_small/D.pickle


52911
./notMNIST_small/E.pickle


...

52911
./notMNIST_small/G.pickle


52911
./notMNIST_small/H.pickle


52911
./notMNIST_small/I.pickle


52911
./notMNIST_small/J.pickle


52911


Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `train_size` as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [9]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (200000, 28, 28) (200000,)
Validation: (10000, 28, 28) (10000,)
Testing: (10000, 28, 28) (10000,)


Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [10]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

---

In [11]:
from numpy import bincount, unique 

print(unique(train_labels))
print(unique(test_labels))
print(unique(valid_labels))
print(bincount(train_labels))
print(bincount(test_labels))
print(bincount(valid_labels))

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]
[20000 20000 20000 20000 20000 20000 20000 20000 20000 20000]
[1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
[1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]


Finally, let's save the data for later reuse:

In [12]:
pickle_file = os.path.join(data_root, 'notMNIST.pickle')

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [13]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 690800506


---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [15]:
import os

import matplotlib.pyplot as plt

def reshape(a):
    return a.reshape(a.shape[0], a.shape[1] * a.shape[2])


def save(output_file, data):
    print("saving data")
    try:
        f = open(output_file, 'wb')
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        f.close()
    except Exception as e:
        print('Unable to save data to', pickle_overlapping_file, ':', e)
        raise


def calculate_overlapping(dataset1, dataset2):
    overlapping = {}
    for index_tr, image_tr in enumerate(dataset1):
        for index_te, image_te in enumerate(dataset2):
            if image_tr == image_te:
                if index_tr not in overlapping:
                    overlapping[index_tr] = []

                overlapping[index_tr].append(index_te)

    return overlapping


train_dataset_hashed = []
test_dataset_hashed = []
valid_dataset_hashed = []
overlapping_test_training = {}
overlapping_test_valid = {}

pickle_overlapping_file = os.path.join(data_root, 'overlapping_info.pickle')

if os.path.exists(pickle_overlapping_file):
    print("Loading data from the pickle file")

    with open(pickle_overlapping_file, 'rb') as f:
        data = pickle.load(f)

        overlapping_test_training = data['overlapping_test_training']
        overlapping_test_valid = data['overlapping_test_valid']
        valid_dataset_hashed = data['valid_dataset_hashed']
        test_dataset_hashed = data['test_dataset_hashed']
        train_dataset_hashed = data['training_dataset_hashed']

else:
    pickle_file = os.path.join(data_root, 'notMNIST.pickle')
    with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)

        print("Starting up")

        train_dataset = letter_set['train_dataset']
        test_dataset = letter_set['test_dataset']
        valid_dataset = letter_set['valid_dataset']

        print("Hashing training dataset")
        for row in reshape(train_dataset):
            train_dataset_hashed.append(hash(row.tostring()))

        print("Hashing test dataset")
        for row in reshape(test_dataset):
            test_dataset_hashed.append(hash(row.tostring()))

        print("Hashing validation dataset")
        for row in reshape(valid_dataset):
            valid_dataset_hashed.append(hash(row.tostring()))

        overlapping_test_training = calculate_overlapping(train_dataset_hashed, test_dataset_hashed)
        print("Overlapping training and test dataset:" + str(len(overlapping_test_training.keys())))

        overlapping_test_valid = calculate_overlapping(test_dataset_hashed, valid_dataset_hashed)
        print("Overlapping test and validation dataset:" + str(len(overlapping_test_valid.keys())))

        data = {
            'overlapping_test_training': overlapping_test_training,
            'overlapping_test_valid': overlapping_test_valid,
            'valid_dataset_hashed': valid_dataset_hashed,
            'test_dataset_hashed': test_dataset_hashed,
            'training_dataset_hashed': train_dataset_hashed
        }

        save(pickle_overlapping_file, data)


Starting up
Hashing training dataset


Hashing test dataset
Hashing validation dataset


Overlapping training and test dataset:3444


Overlapping test and validation dataset:196
saving data


In [17]:
import os
import matplotlib.pyplot as plt
from scipy.spatial import distance

def reshape(a):
    return a.reshape(a.shape[0], a.shape[1] * a.shape[2])


def near_overlapping(dataset1, dataset2):
    overlapping = {}
    for index_1, image_1 in enumerate(dataset1):
        for index_2, image_2 in enumerate(dataset2):
            if distance.euclidean(image_1, image_2) < 0.3:
                if index_1 not in overlapping:
                    overlapping[index_1] = []

                overlapping[index_1].append(index_2)
    return overlapping


pickle_near_overlapping_file = os.path.join(data_root, 'near_overlapping_info.pickle')
near_overlapping_test_training = {}
near_overlapping_test_valid = {}

if os.path.exists(pickle_near_overlapping_file):
    print("Loading data from the pickle file")

    with open(pickle_near_overlapping_file, 'rb') as f:
        data = pickle.load(f)

        near_overlapping_test_training = data['near_overlapping_test_training']
        near_overlapping_test_valid = data['near_overlapping_test_valid']

else:
    print("Starting up")

    near_overlapping_test_training = near_overlapping(train_dataset_hashed, test_dataset_hashed)
    print("Near Overlapping of training and test dataset:" + str(len(overlapping_test_training.keys())))

    near_overlapping_test_valid = near_overlapping(test_dataset_hashed, valid_dataset_hashed)
    print("Near Overlapping test and validation dataset:" + str(len(overlapping_test_valid.keys())))

    data = {
        'near_overlapping_test_training': near_overlapping_test_training,
        'near_overlapping_test_valid': near_overlapping_test_valid
    }

    save(pickle_near_overlapping_file, data)


Starting up


KeyboardInterrupt: 

In [19]:
# print(overlapping_test_training)
for idx, train_index in enumerate(overlapping_test_training):
    plt.subplot(2, 4, idx + 1)
    plt.imshow(train_dataset[train_index])
    test_list = overlapping_test_training[train_index]
    for tidx, test_image in enumerate(test_list[:4]):
        test_imge = test_dataset[i]
        plt.subplot(2, 4, tidx + 1)
        plt.axis('off')
        plt.imshow(test_dataset[test_image])


/Users/lfoppiano/anaconda3/envs/tensorflow/lib/python3.6/site-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


NameError: name 'i' is not defined

In [ ]:
# Sanitize the training data 



---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [26]:
from sklearn.linear_model import LogisticRegression as lr

print("Start the learning phase")

# We try several values for C 
for n in [50, 100, 1000, 5000]:
    for c in [1.0, 0.5, 0.2, 0.1, 0.01]:
        linear_regression = lr(multi_class="multinomial", solver="sag", C=c,
                                                                    max_iter=1000)
        model = linear_regression.fit(reshape(train_dataset)[:n], train_labels[:n])
        score = model.score(reshape(test_dataset), test_labels)
        print("n= " + str(n) + ", C= " + str(c) + ", score= " + str(score))


Start the learning phase


/Users/lfoppiano/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


n= 50, C= 1.0, score= 0.6412


n= 50, C= 0.5, score= 0.6327


n= 50, C= 0.2, score= 0.6202


...

n= 5000, C= 1.0, score= 0.8425


n= 5000, C= 0.5, score= 0.8566


n= 5000, C= 0.2, score= 0.8679


n= 5000, C= 0.1, score= 0.8741


n= 5000, C= 0.01, score= 0.8805


We select C = 0.01


In [27]:
from sklearn.linear_model import LogisticRegression as lr

for n in [50, 100, 1000, 5000]:
    linear_regression = lr(multi_class="multinomial", solver="sag", C=0.01,
                                                                max_iter=1000)
    model = linear_regression.fit(reshape(train_dataset)[:n], train_labels[:n])
    score = model.score(reshape(test_dataset), valid_labels)
    print("n= " + str(n) + ", C= " + str(0.01) + ", score= " + str(score))


n= 50, C= 0.01, score= 0.096


n= 100, C= 0.01, score= 0.0972


n= 1000, C= 0.01, score= 0.097


n= 5000, C= 0.01, score= 0.0972


In [28]:
import random

pretty_labels = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J'}

def disp_sample_dataset(dataset, labels):
    items = random.sample(range(len(labels)), 8)
    for i, item in enumerate(items):
        print(item)
        plt.subplot(2, 4, i + 1)
        plt.axis('off')
        plt.title(pretty_labels[int(round(labels[item]))])
        plt.imshow(dataset[item])

# disp_sample_dataset(valid_dataset, predictions)


NameError: name 'predictions' is not defined